In [ ]:
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()

True
39
39 39


In [70]:
comparison_df = pd.read_csv("../data/processed/model_comparison_metrics.csv")
comparison_df.head()

,model,class,precision,recall,f1_score,support
0,Baseline,0,0.999385,0.999771,0.999578,56864.0
1,Baseline,1,0.828947,0.642857,0.724138,98.0
2,Balanced,0,0.999856,0.975556,0.987556,56864.0
3,Balanced,1,0.060811,0.918367,0.114068,98.0


In [71]:
fraud_df = comparison_df[comparison_df["class"].isin([1, "1"])].copy()
fraud_df

,model,class,precision,recall,f1_score,support
1,Baseline,1,0.828947,0.642857,0.724138,98.0
3,Balanced,1,0.060811,0.918367,0.114068,98.0


In [72]:
results_txt = fraud_df.to_string(index=False)
results_txt

'   model  class  precision   recall  f1_score  support\nBaseline      1   0.828947 0.642857  0.724138     98.0\nBalanced      1   0.060811 0.918367  0.114068     98.0'

In [73]:
prompt = PromptTemplate(
    input_variables=["results"],
    template=""" 
    Você é um cientista de dados especializado em detecção de fraudes.

    Abaixo estão métricas (precision, recall, f1_score, support) para a CLASSE FRAUDE (classe 1)
    de dois modelos: Baseline e Balanced.

    Resultados:
    {results}

    Tarefas:
    1) Explique, em português claro, o desempenho de cada modelo (foco em precision vs recall).
    2) Qual modelo você recomendaria para um sistema de alerta de fraude e por quê?
    3) Quais riscos e trade-offs existem com o modelo recomendado?
    4) Sugira 2 melhorias práticas para a próxima iteração do projeto.
    Responda em tópicos curtos.  
"""
)

In [74]:
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0)

final_prompt = prompt.format(results=results_txt)
response = llm.invoke(final_prompt)

print(response.content)

[{'type': 'text', 'text': 'Como cientista de dados, aqui está a análise técnica dos resultados:\n\n### 1) Explicação do Desempenho\n*   **Baseline:** É um modelo "conservador". Possui alta **Precision (83%)**, o que significa que quando ele aponta fraude, ele geralmente está certo. Porém, tem um **Recall baixo (64%)**, deixando passar 36% das fraudes reais (falsos negativos).\n*   **Balanced:** É um modelo "agressivo". Prioriza o **Recall (92%)**, detectando quase todas as fraudes. Contudo, sua **Precision é baixíssima (6%)**, o que significa que para cada 100 alertas gerados, apenas 6 são fraudes reais e 94 são alarmes falsos.\n\n### 2) Modelo Recomendado\n*   **Recomendação:** O modelo **Balanced** (com ressalvas operacionais).\n*   **Por que?** Em detecção de fraude, o custo financeiro de uma fraude não detectada (falso negativo) costuma ser muito superior ao custo de revisar uma transação legítima. O modelo Balanced garante que o prejuízo financeiro direto seja minimizado ao captur

In [75]:
def extract_text_from_parts(content) -> str:
    if isinstance(content, list):
        texts = []
        for part in content:
            if isinstance(part, dict) and part.get("type") == "text":
                texts.append(part.get("text", ""))
        return "\n".join(t for t in texts if t)
    return str(content)

In [76]:
clean_text = extract_text_from_parts(response.content)
print(clean_text)

Como cientista de dados, aqui está a análise técnica dos resultados:

### 1) Explicação do Desempenho
*   **Baseline:** É um modelo "conservador". Possui alta **Precision (83%)**, o que significa que quando ele aponta fraude, ele geralmente está certo. Porém, tem um **Recall baixo (64%)**, deixando passar 36% das fraudes reais (falsos negativos).
*   **Balanced:** É um modelo "agressivo". Prioriza o **Recall (92%)**, detectando quase todas as fraudes. Contudo, sua **Precision é baixíssima (6%)**, o que significa que para cada 100 alertas gerados, apenas 6 são fraudes reais e 94 são alarmes falsos.

### 2) Modelo Recomendado
*   **Recomendação:** O modelo **Balanced** (com ressalvas operacionais).
*   **Por que?** Em detecção de fraude, o custo financeiro de uma fraude não detectada (falso negativo) costuma ser muito superior ao custo de revisar uma transação legítima. O modelo Balanced garante que o prejuízo financeiro direto seja minimizado ao capturar 92% dos casos.

### 3) Riscos e 

In [77]:
ai_report = clean_text

with open("../data/processed/ai_conclusions.md", "w", encoding="utf-8") as f:
    f.write("# Conclusões da IA sobre os models\n\n")
    f.write(ai_report)

print("Arquivo salvo!")

Arquivo salvo!
